In [ ]:
import os

import numpy as np
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

from models import VisRank
from utils.data import extract_embedding
from utils.metrics import (
    auc_exact,
    nDCG,
    precision,
    recall,
    reciprocal_rank,
)


# Parameters
DATASET = "Wikimedia"
assert DATASET in ["UGallery", "Wikimedia", "Pinterest", "Tradesy"]
FEATURE_EXTRACTOR = "resnet50"
assert FEATURE_EXTRACTOR in ["resnet50"]
FEATURE_EXTRACTOR_VERSION = "imagenet"
assert FEATURE_EXTRACTOR_VERSION in ["imagenet", "places365"]


In [ ]:
# Mode
MODE_PROFILE = "profile"

# Paths (general)
EMBEDDING_PATH = os.path.join("data", DATASET, f"{DATASET.lower()}_embedding-{FEATURE_EXTRACTOR}.npy")
EVALUATION_PATH = os.path.join("data", DATASET, f"{MODE_PROFILE}-evaluation.csv")

# Paths (images)
IMAGES_DIR = None
if DATASET == "UGallery":
    IMAGES_DIR = os.path.join("/", "mnt", "workspace", "Ugallery", "images")
elif DATASET == "Wikimedia":
    IMAGES_DIR = os.path.join("/", "mnt", "data2", "wikimedia", "images", "img")
elif DATASET == "Pinterest":
    IMAGES_DIR = os.path.join("/", "mnt", "data2", "pinterest_iccv", "images")
elif DATASET == "Tradesy":
    print("Tradesy dataset not supported at the moment.")


In [ ]:
# Load evaluation dataframe
print("\nLoad evaluation dataframe")
evaluation_df = pd.read_csv(EVALUATION_PATH)
# Transform lists from str to int
string_to_list = lambda s: list(map(int, s.split()))
evaluation_df["profile"] = evaluation_df["profile"].apply(
    lambda s: string_to_list(s) if isinstance(s, str) else s,
)
evaluation_df["predict"] = evaluation_df["predict"].apply(
    lambda s: string_to_list(s) if isinstance(s, str) else s,
)
# Group evaluations by profile and user
evaluation_df["profile"] = evaluation_df["profile"].map(tuple)
evaluation_df = evaluation_df.groupby(["profile", "user_id"]).agg({"predict": sum}).reset_index()
evaluation_df["profile"] = evaluation_df["profile"].map(list)
print(f">> Evaluation: {evaluation_df.shape}")


In [ ]:
evaluation_df["profile"] = evaluation_df["profile"].map(tuple)


## Mass testing for embeddings

In [ ]:
# Predict all
# If True, ranks every item including already consumed items
# If False, ranks ALL - PROFILE (consumed) + PREDICT (ground truth)
PREDICT_ALL = False


In [ ]:
import random


COMBINATIONS = [
    (p1, p2, p3, p4)
    for p1 in range(4)  # UNIT_CRITERIA
    for p2 in range(2)  # UNIQUE_DETECTORS_ONLY
    for p3 in range(5)  # LAYER_CRITERIA
    for p4 in range(2)  # SAME_CONCEPT_UNITS_CRITERIA
]
# random.shuffle(COMBINATIONS)


In [ ]:
ROWS = list()

In [ ]:
for (p1, p2, p3, p4) in tqdm(COMBINATIONS):
    print(p1, p2, p3, p4)
    
    # Check if embedding exists
    EMBEDDING_PATH = os.path.join(
        "..", "..",
        "embeddings",
        "{}_{}_{}".format(FEATURE_EXTRACTOR, FEATURE_EXTRACTOR_VERSION, DATASET.lower()),
        "{}{}{}{}.npy".format(p1, p2, p3, p4),
    )
    if not os.path.exists(EMBEDDING_PATH):
        print("Embedding not found:", p1, p2, p3, p4)
        continue

        
    # Load embedding from file
    print(f"\nLoading embedding from file... ({EMBEDDING_PATH})")
    embedding = np.load(EMBEDDING_PATH, allow_pickle=True)

    # Extract features and "id2index" mapping
    features, _, item_index2fn = extract_embedding(embedding, verbose=True)
    del embedding  # Release some memory

    # Model initialization
    model = VisRank(
        features,  # Embedding
        similarity_method=cosine_similarity,  # Similarity measure
    )
    print("\nModel ready...")

    # Metrics
    N_EVALS = len(evaluation_df.index)
    # Area Under the Curve (AUC)
    AUC = np.zeros(N_EVALS, dtype=float)
    # Reciprocal Rank (RR)
    RR = np.zeros(N_EVALS, dtype=float)
    # Recall
    R20 = np.zeros(N_EVALS, dtype=float)
    R100 = np.zeros(N_EVALS, dtype=float)
    R200 = np.zeros(N_EVALS, dtype=float)
    # Precision
    P20 = np.zeros(N_EVALS, dtype=float)
    P100 = np.zeros(N_EVALS, dtype=float)
    P200 = np.zeros(N_EVALS, dtype=float)
    # Normalized discounted cumulative gain (nDCG)
    N20 = np.zeros(N_EVALS, dtype=float)
    N100 = np.zeros(N_EVALS, dtype=float)
    N200 = np.zeros(N_EVALS, dtype=float)
    PROFILE_SIZES = np.zeros(N_EVALS, dtype=int)
    N_ITEMS = len(features)

    # Evaluation loop
    for i, row in tqdm(enumerate(evaluation_df.itertuples()), total=len(evaluation_df.index)):
        # Load data into tensors
        profile = np.array(row.profile)
        user_id = int(row.user_id)
        predict = row.predict
        # Prediction
        indexes, _ = model.most_similar_to_profile(profile, k=None, method="maximum", include_consumed=True)
        if not PREDICT_ALL:
            indexes = np.delete(
                indexes,
                np.where(np.isin(indexes, profile) & ~np.isin(indexes, predict)),
            )
        # Ranking
        pos_of_evals = torch.Tensor(np.where(np.isin(indexes, predict))).flatten()
        # Store metrics
        AUC[i] = auc_exact(pos_of_evals, N_ITEMS)
        RR[i] = reciprocal_rank(pos_of_evals)
        R20[i] = recall(pos_of_evals, 20)
        P20[i] = precision(pos_of_evals, 20)
        N20[i] = nDCG(pos_of_evals, 20)
        R100[i] = recall(pos_of_evals, 100)
        P100[i] = precision(pos_of_evals, 100)
        N100[i] = nDCG(pos_of_evals, 100)
        R200[i] = recall(pos_of_evals, 200)
        P200[i] = precision(pos_of_evals, 200)
        N200[i] = nDCG(pos_of_evals, 200)
        PROFILE_SIZES[i] = len(row.profile)
        
    ROWS.append(
        (
            p1, p2, p3, p4,
            AUC.mean(), RR.mean(),
            R20.mean(), P20.mean(), N20.mean(),
            R100.mean(), P100.mean(), N100.mean(),
            R200.mean(), P200.mean(), N200.mean(),
        )
    )
    print(ROWS[-1])

# Wikimedia: ~1.25 hrs


In [ ]:
RESULTS = pd.DataFrame(
    ROWS,
    columns=[
        "p1", "p2", "p3", "p4",
        "AUC", "RR",
        "R20", "P20", "N20",
        "R100", "P100", "N100",
        "R200", "P200", "N200",
    ],
)


In [ ]:
RESULTS_PATH = os.path.join("..", "..", "embeddings", "results", f"{FEATURE_EXTRACTOR}_{FEATURE_EXTRACTOR_VERSION}_{DATASET.lower()}.csv")
RESULTS.to_csv(RESULTS_PATH)
print(RESULTS_PATH)
